<a href="https://colab.research.google.com/github/eyash24/Data-Science/blob/main/NLP_101_ML_Language_Translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning Language Translator Using Sequence to Sequence NN
Following Krish Naik  video: https://youtu.be/f-JCCOHwx1c?si=TR8moBCoRkNQCdMI
<br>
And the article blog: https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html
<br>
And this article: https://keras.io/examples/nlp/lstm_seq2seq/

In [ ]:
! wget https://www.manythings.org/anki/fra-eng.zip

--2024-07-03 19:20:48--  https://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7943074 (7.6M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   7.57M  34.0MB/s    in 0.2s    

2024-07-03 19:20:48 (34.0 MB/s) - ‘fra-eng.zip’ saved [7943074/7943074]



In [ ]:
! unzip fra-eng.zip

Archive:  fra-eng.zip
  inflating: _about.txt              
  inflating: fra.txt                 


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np
import pandas as pd


In [ ]:
batch_size = 64
epochs = 100
latent_dim = 256
num_samples = 10000


In [ ]:
# data path
data_path = 'fra.txt'

In [ ]:
# Vectorising the data
input_texts = []
target_texts = []
input_char = set()
target_char = set()

with open(data_path, 'r', encoding='utf-8') as f:
  lines = f.read().split('\n')

for line in lines[: min(num_samples, len(lines)-1)]:
  input_txt, target_txt, _ = line.split('\t')

  target_txt = '\t' + target_txt + '\n'
  input_texts.append(input_txt)
  target_texts.append(target_txt)

  for char in input_txt:
    if char not in input_char:
      input_char.add(char)

  for char in target_txt:
    if char not in target_char:
      target_char.add(char)

In [ ]:
input_char = sorted(list(input_char))
target_char = sorted(list(target_char))
num_encoder_tokens = len(input_char)
num_decoder_tokens = len(target_char)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [ ]:
print("No of samples: ", len(input_texts))
print("No of unique input tokens: ", num_encoder_tokens)
print("No of unique output tokens: ", num_decoder_tokens)
print('Max seq len for input: ', max_encoder_seq_length)
print('Max seq len for output: ', max_decoder_seq_length)

No of samples:  232736
No of unique input tokens:  90
No of unique output tokens:  114
Max seq len for input:  292
Max seq len for output:  351


In [ ]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_char)]
)

target_token_index = dict(
    [(char, i) for i, char in enumerate(target_char)]
)

In [ ]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32'
)

decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),  dtype='float32'
)

decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),  dtype='float32'
)

In [ ]:
# one hot encoding

for i, (input_txt, target_txt) in enumerate(zip(input_texts, target_texts)):
  for t, char in enumerate(input_txt):
    encoder_input_data[i, t, input_token_index[char]] = 1.0
  encoder_input_data[i, t+1: , input_token_index[' ']] = 1.0

  for t, char in enumerate(target_txt):
    decoder_target_data[i, t, target_token_index[char]] = 1.0

    if t > 0:
      decoder_target_data[i, t-1, target_token_index[char]] = 1.0
  decoder_input_data[i, t+1: , target_token_index[' ']] = 1.0
  decoder_target_data[i, t: , target_token_index[' ']] = 1.0

In [ ]:
# define an input sequence and process it
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state = True)
encoder_output, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]

In [ ]:
# setup decoder using encoder_state as initial state
decoder_inputs = Input(shape=(None, num_decoder_tokens))

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
# creating the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(
    optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy']
)


In [ ]:
model.fit([encoder_input_data, decoder_input_data],
          decoder_target_data,
          batch_size = batch_size,
          epochs=epochs,
          validation_split=0.2
          )

In [ ]:
# Define sampling models
encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)